In [60]:
import pandas as pd 
import numpy as np 
import seaborn as sns 
sns.set(style='white',palette='muted',color_codes=True)
import matplotlib.pyplot as plt 
plt.style.use('ggplot')
import plotly.express as px

In [61]:
from sklearn.impute import SimpleImputer 
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LinearRegression,LogisticRegression 
from sklearn.preprocessing import StandardScaler,MinMaxScaler,OneHotEncoder,LabelEncoder,OrdinalEncoder 
from sklearn.compose import ColumnTransformer 
from sklearn.metrics import silhouette_score,accuracy_score,r2_score,confusion_matrix,mean_squared_error 
from sklearn.ensemble import RandomForestRegressor,RandomForestClassifier 
from sklearn.pipeline import Pipeline 
from sklearn.neighbors import KNeighborsClassifier,KNeighborsRegressor 
from sklearn.ensemble import RandomForestClassifier,RandomForestRegressor 
from sklearn.cluster import KMeans 
from sklearn.decomposition import PCA 

In [62]:
data = pd.read_csv(r'c:\Github\Fullstack-Data-Analyst\Learning\the_data\data-lab-4-heart.csv')
data.head(1)

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
